In [1]:
import pandas as pd
import numpy as np
from data_acquisition_and_processing import *
from training_models import *

In [2]:
faults_filepath = '../data/J1939Faults.csv'
diagnostics_filepath = '../data/VehicleDiagnosticOnboardData.csv'

In [3]:
train_df, test_df = clean_data(faults_filepath, diagnostics_filepath)
train_df.head()

Merging faults and diagnostics data...


c:\Users\irela\Documents\NSS_Projects\big-g-haulin-oats\notebooks\data_acquisition_and_processing.py:9: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv(faults_filepath)


Removing service locations and unnecessary columns...
Number of rows before removing service locations:
1187335
Number of rows after removing service locations:
1051146
Unnecessary columns removed:
['RecordID', 'ESS_Id', 'eventDescription', 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'active', 'activeTransitionCount', 'faultValue', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp']
Data after removing service locations and unnecessary columns:
       EventTimeStamp   spn fmi EquipmentID  AcceleratorPedal  \
0 2015-02-21 10:47:13   111  17        1439               0.0   
1 2015-02-21 11:34:34   629  12        1439               NaN   
2 2015-02-21 11:35:31  1807   2        1369               NaN   
3 2015-02-21 11:35:33  1807   2        1369               NaN   
4 2015-02-21 11:39:41  4364  17        1674               NaN   

   BarometricPressure  CruiseControlActive  CruiseControlSetSpeed  \
0               14.21         

,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
1001106,2018-07-20 09:31:33,1231,0105406655,-0.026563,-1.839029,0.370184,-0.644781,0.539284,0.318816,0.396278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001107,2018-07-20 09:42:30,1231,0105406655,-0.026563,-1.839029,0.370184,-0.644781,0.539284,0.318816,0.396278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
358800,2016-01-31 07:12:25,629,105301976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
927313,2018-01-26 18:58:08,37,105338729,0.316419,-2.466096,0.041213,-0.513425,-1.129315,0.257644,-0.956231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
927314,2018-01-26 18:59:36,37,105338729,0.316419,-2.466096,0.041213,-0.513425,-1.129315,0.257644,-0.956231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
prepare_data_for_model(train_df, test_df, target_filename='ffill_05_09')

Processing target: HoursUntilNextDerate
Saved HoursUntilNextDerate dataframes for training and testing in data/data_ready_for_models/
Processing target: DerateInNextTwoHours
Saved DerateInNextTwoHours dataframes for training and testing in data/data_ready_for_models/
Processing target: DerateInNextTwentyFourHours
Saved DerateInNextTwentyFourHours dataframes for training and testing in data/data_ready_for_models/


In [5]:
from sklearn.tree import DecisionTreeClassifier

train_data_file_path = '../data/data_ready_for_models/DerateInNextTwentyFourHours_ffill_05_09_train.csv'
test_data_file_path = '../data/data_ready_for_models/DerateInNextTwentyFourHours_ffill_05_09_test.csv'
model = DecisionTreeClassifier(random_state=42)
model_name = 'initial_decision_tree_classifier'

predictions_filepath = f'../data/model_output/DerateInNextTwentyFourHours_ffill_05_09_{model_name}.csv'

In [6]:
test_df = pd.read_csv(test_data_file_path)
test_df.head()

,DerateInNextTwentyFourHours,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
0,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,2019-01-08 19:42:39,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2019-01-08 19:42:39,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,2019-01-16 19:41:28,37,105338729,-0.140890,-1.760107,0.260527,-0.546264,-1.077171,0.077285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
test_df[test_df['FullDerate']==1]

,DerateInNextTwentyFourHours,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
652,1,2019-01-13 21:21:37,5246,1698,0.202092,1.212431,0.699154,-0.808977,-3.006489,0.117365,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
653,1,2019-01-14 13:54:56,5246,1698,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3937,1,2019-02-20 19:21:49,5246,1739,0.487910,1.663243,0.205698,-0.382068,0.226422,-0.432147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3938,1,2019-02-21 05:47:45,5246,1739,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3939,1,2019-02-21 07:17:47,5246,1739,0.316419,1.663432,0.534669,-0.743299,-1.025027,0.164828,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103847,1,2020-01-13 12:15:04,5246,302,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103848,1,2020-01-13 13:18:31,5246,302,0.373583,-0.151485,-0.671557,-0.743299,2.520745,-0.707429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103849,1,2020-01-22 21:15:03,5246,302,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103850,1,2020-02-14 11:21:54,5246,302,0.487910,-0.144037,-4.619207,0.635945,4.606494,-3.161770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
start_mlflow_run(train_data_file_path, test_data_file_path, predictions_filepath, model)

2025/05/09 16:31:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
c:\Users\irela\Documents\NSS_Projects\big-g-haulin-oats\notebooks\training_models.py:47: DtypeWarning: Columns (3,13) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_data_file_path)
2025/05/09 16:32:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\irela\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns 

Test precision: 0.1388888888888889
Test recall: 0.25157232704402516
Test F1 score: 0.1789709172259508
Test accuracy: 0.9930251342234048
Model training and evaluation completed successfully.


In [9]:
predictions_df = pd.read_csv(predictions_filepath)
predictions_df.head()

,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [10]:
print(len(predictions_df))
print(len(test_df.dropna()))

merged = pd.merge(predictions_df, test_df.dropna().reset_index(), left_index=True, right_index=True)
merged.head()

105235
105235


,Actual,Predicted,index,DerateInNextTwentyFourHours,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
0,0,0,0,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0,2,0,2019-01-08 19:42:39,639,105338729,-0.083727,-1.773036,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,3,0,2019-01-08 19:42:39,639,105338729,-0.083727,-1.773036,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,0,4,0,2019-01-16 19:41:28,37,105338729,-0.140890,-1.760107,0.260527,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
merged[merged['Actual']==merged['DerateInNextTwentyFourHours']]

,Actual,Predicted,index,DerateInNextTwentyFourHours,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
0,0,0,0,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0,2,0,2019-01-08 19:42:39,639,105338729,-0.083727,-1.773036,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,3,0,2019-01-08 19:42:39,639,105338729,-0.083727,-1.773036,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,0,4,0,2019-01-16 19:41:28,37,105338729,-0.140890,-1.760107,0.260527,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105230,0,0,105375,0,2019-10-09 23:32:35,639,310,0.202092,-0.214855,0.315355,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105231,0,0,105376,0,2019-11-12 00:16:41,789,310,0.144928,-0.192790,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105232,0,0,105377,0,2019-11-12 00:16:41,789,310,0.144928,-0.192790,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105233,0,0,105378,0,2019-11-12 00:24:39,639,310,0.144928,-0.192790,0.479841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
merged[merged['FullDerate']==1]

,Actual,Predicted,index,DerateInNextTwentyFourHours,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
650,1,1,652,1,2019-01-13 21:21:37,5246,1698,0.202092,1.212431,0.699154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3928,1,1,3937,1,2019-02-20 19:21:49,5246,1739,0.487910,1.663243,0.205698,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3929,1,1,3939,1,2019-02-21 07:17:47,5246,1739,0.316419,1.663432,0.534669,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6737,1,1,6754,1,2019-01-29 22:35:39,5246,1758,0.487910,0.859555,0.315355,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6738,1,1,6756,1,2019-01-31 21:23:21,5246,1758,0.430746,0.866320,0.205698,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6739,1,1,6758,1,2019-02-01 02:42:48,5246,1758,0.316419,0.867728,0.425012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6740,1,1,6760,1,2019-02-05 19:59:02,5246,1758,0.087764,0.876292,0.315355,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6741,1,1,6762,1,2019-02-06 02:31:49,5246,1758,0.144928,0.877028,0.370184,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7934,1,1,7956,1,2019-03-26 07:37:57,5246,1768,0.030601,1.497255,-1.329499,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11531,1,1,11555,1,2019-05-29 19:41:57,5246,1797,0.087764,0.400797,-0.013616,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
TP = ((predictions_df['Actual'] == 1) & (predictions_df['Predicted'] == 1)).sum()
FP = ((predictions_df['Actual'] == 0) & (predictions_df['Predicted'] == 1)).sum()
FN = ((predictions_df['Actual'] == 1) & (predictions_df['Predicted'] == 0)).sum()
TN = ((predictions_df['Actual'] == 0) & (predictions_df['Predicted'] == 0)).sum()

print(f'Precision: {TP / (TP + FP) if (TP + FP) > 0 else 0}')
print(f'TP = {TP}')
print(f'FP = {FP}')

Precision: 0.1388888888888889
TP = 80
FP = 496


In [15]:
from sklearn.metrics import classification_report, confusion_matrix

y_test = predictions_df['Actual']
y_pred = predictions_df['Predicted']

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["No Derate", "Derate"]))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred, labels=[1, 0]))  # 1 is derate and 0 is no derate

Classification Report:
              precision    recall  f1-score   support

   No Derate       1.00      1.00      1.00    104917
      Derate       0.14      0.25      0.18       318

    accuracy                           0.99    105235
   macro avg       0.57      0.62      0.59    105235
weighted avg       1.00      0.99      0.99    105235

Confusion Matrix:
[[    80    238]
 [   496 104421]]
